In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!tar zxvf checkpoints.tar.gz

In [2]:
!tar zxvf data.tar.gz
!tar zxvf deepcomedy.tar.gz
!tar zxvf nlgpoetry.tar.gz

data/
data/orlando.txt
data/divina_textonly.txt
data/divina.txt
data/divina_syll_textonly.txt
data/orlando-textonly.txt
data/divina_syll.txt
data/.ipynb_checkpoints/
data/.ipynb_checkpoints/orlando-checkpoint.txt
data/.ipynb_checkpoints/orlando-textonly-checkpoint.txt
data/.ipynb_checkpoints/divina_textonly-checkpoint.txt
data/.ipynb_checkpoints/divina_syll-checkpoint.txt
data/.ipynb_checkpoints/divina-checkpoint.txt
data/.ipynb_checkpoints/divina_syll_textonly-checkpoint.txt
deepcomedy/
deepcomedy/util/
deepcomedy/util/predicate.py
deepcomedy/util/__pycache__/
deepcomedy/util/__pycache__/predicate.cpython-37.pyc
deepcomedy/util/__pycache__/__init__.cpython-37.pyc
deepcomedy/util/__init__.py
deepcomedy/util/.ipynb_checkpoints/
deepcomedy/util/.ipynb_checkpoints/predicate-checkpoint.py
deepcomedy/models/
deepcomedy/models/layers.py
deepcomedy/models/transformer.py
deepcomedy/models/__pycache__/
deepcomedy/models/__pycache__/layers.cpython-37.pyc
deepcomedy/models/__pycache__/__init__.cp

In [3]:
from nlgpoetry.hyphenation import *

In [4]:
from deepcomedy.preprocessing import load_verses
from deepcomedy.models.transformer import *
from deepcomedy.util.predicate import predicate
import tqdm

In [5]:
import re
import tensorflow as tf

In [6]:
%load_ext autoreload
%autoreload 2

### TODO
potremmo usare il codice del prof per trovare le discordanze dovute a sinalefe non gestita.

In [7]:
_, verses, tokenizer = load_verses('./data/divina_textonly.txt', char_level=True, pad=True)

In [ ]:
orlando_path = 'data/orlando.txt'
orlando = open(orlando_path, "rb").read().decode(encoding="utf-8")
orlando = orlando.split('\n')[44:53579]

In [ ]:
@predicate
def is_not_empty(string):
    """
    Checks string is not empty
    """
    return string is not None and string != ''

@predicate
def is_not_number(string):
    try:
        int(string)
        return False
    except:
        return True

@predicate
def is_not_chapter(string):
    return not re.match(r'CANTO .*', string)

In [ ]:
orlando_textonly = list(map(lambda x: x.strip(), filter(is_not_empty & is_not_number & is_not_chapter, orlando)))
orlando_syll = list(map(hyphenation, orlando_textonly))

In [ ]:
orlando_syll

In [ ]:
orlando_textonly

In [ ]:
set(''.join(orlando))

In [8]:
num_layers = 6
d_model = 256
dff = 1024
num_heads = 8
dropout_rate = 0.1

vocab_size = 82

In [9]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate,
)
learning_rate = TransformerCustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)

In [18]:
ckpt = tf.train.Checkpoint(
    transformer=transformer, optimizer=optimizer
)

checkpoint_manager = tf.train.CheckpointManager(
    ckpt, '/content/drive/MyDrive/checkpoints/train', max_to_keep=5
)

# if a checkpoint exists, restore the latest checkpoint.
if checkpoint_manager.latest_checkpoint:
    ckpt.restore(checkpoint_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [19]:
def evaluate(sentence, max_length=200):

    encoder_input = tf.expand_dims(sentence, 0)
    output = tf.convert_to_tensor([tokenizer.word_index["^"]])
    output = tf.expand_dims(output, 0)
    result = ""

    for i in range(max_length):
        
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output
        )

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(
            encoder_input,
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask,
        )

        # select the last character from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.argmax(predictions, axis=-1)

        # concatenate the predicted_id to the output which is given to the decoder as its input.
        output = tf.concat(
            [tf.cast(output, dtype=tf.int32), tf.cast(predicted_id, dtype=tf.int32)],
            axis=-1,
        )
        result += tokenizer.index_word[predicted_id.numpy()[0][0]] + " "

        # return the result if the predicted_id is equal to the end token
        if predicted_id == tokenizer.word_index["$"]:
            break

    # output.shape (1, tokens)

    return result

In [ ]:
neural_syll = list(map(evaluate, verses))

In [ ]:
neural_syll = []

for i, verse in tqdm.tqdm(enumerate(verses), total=len(verses)):
    neural_syll.append(evaluate(verse))


  0%|          | 0/14233 [00:00<?, ?it/s]